In [1]:
import pandas as pd
from soil_fertility.components.data_ingestion import DataIngestion
from soil_fertility.components.data_transformation import DataTransformation

In [2]:
obj = DataIngestion()
part,train_path,test_path= obj.init_ingestion(
    "/home/redha/Documents/projects/NLP/datamining project/Soil-Fertility/data/Dataset1.csv"
)


In [4]:
train_path

'/home/redha/Documents/projects/NLP/datamining project/Soil-Fertility/artifacts/raw/1/train.csv'

In [5]:
test_path

'/home/redha/Documents/projects/NLP/datamining project/Soil-Fertility/artifacts/raw/1/test.csv'

In [7]:
tranformation_obj = DataTransformation()
preprocessor,processed_train_path,processed_test_path = tranformation_obj.transform(
    train_path,test_path,target="Fertility"
)

ValueError: too many values to unpack (expected 3)